In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ../src/

/tf/notebooks/src


## Intialization

### GPU

In [3]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### Packages

In [11]:
try:
    import pandas
except:
    !pip install pandas iterative-stratification nlpaug==0.0.20 tqdm click tensorflow_probability==0.11.1 tf2_resnets tensorflow_addons==0.11.1 image-classifiers==0.2.2

### Imports

In [8]:
import os
import random
import logging
import warnings

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt

from dataloader import BalancedMelSampler, MelSampler, convert_csv_to_dict_for_dataloader
from losses import NpairsLoss
from metrics import TFLWLRAP
from split_data import get_split
from train import get_model, get_callbacks, get_lr_metric
from models import NUM_FRAMES, Classifier, DeepMetricLearning

In [9]:
from params import *
from utils.logger import prepare_log_folder, create_logger

### Setup

In [7]:
os.environ["TF_DETERMINISTIC_OPS"] = "1"
os.environ["SM_FRAMEWORK"] = "tf.keras"

physical_devices = tf.config.list_physical_devices("GPU")
for i in range(len(physical_devices)):
    tf.config.experimental.set_memory_growth(physical_devices[i], True)
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

## Main

In [8]:
def get_lr(epoch, lr, min_lr=1e-5, max_lr=1e-3, epochs=30, warmup_prop=0.1):   
    if epoch <= epochs * warmup_prop:
        return min_lr + max_lr - (max_lr * (epochs * warmup_prop - epoch) / (epochs * warmup_prop))
    else:
        return min_lr + max_lr - (max_lr * (epoch - epochs * warmup_prop) / (epochs - epochs * warmup_prop))

In [9]:
def main(fold_idx, saved_path, pretrained_path, pretrained_with_contrastive):
    os.makedirs(os.path.join(saved_path, f"fold{fold_idx}"), exist_ok=True)
    pretrained_with_contrastive = bool(pretrained_with_contrastive)

    print(' -> Preparing Data \n')
    
    train_data = pd.read_csv("../data/new_train_tp.csv")
    train_index, val_index = get_split(fold=fold_idx)
    fold_train_dict = convert_csv_to_dict_for_dataloader(train_data.iloc[train_index])
    fold_valid_dict = convert_csv_to_dict_for_dataloader(train_data.iloc[val_index])

    balanced_train_data_loader = BalancedMelSampler(
        fold_train_dict,
        batch_size=32,
        max_length=NUM_FRAMES,
        is_train=True,
        n_classes=24,
        use_cutmix=True,
        cache=True,
        n_classes_in_batch=8,
        shuffle_aug=False,
    )

    valid_data_loader = MelSampler(
        fold_valid_dict,
        batch_size=balanced_train_data_loader.batch_size,
        n_classes=balanced_train_data_loader.n_classes,
        cache=True,
        max_length=NUM_FRAMES,
        is_train=False,
        use_cutmix=False,
        shuffle_aug=balanced_train_data_loader.shuffle_aug,
    )
    
#     return 0

    print(' -> Preparing Model \n')
    model = get_model(
        saved_path=saved_path,
        pretrained_with_contrastive=pretrained_with_contrastive,
        pretrained_path=pretrained_path,
    )
    model._build()
    
    scheduler = tfa.optimizers.Triangular2CyclicalLearningRate(
        initial_learning_rate=1e-4,
        maximal_learning_rate=1e-3,
        step_size=15 * 20,
    )
    
    optimizer = tf.keras.mixed_precision.experimental.LossScaleOptimizer(
        tfa.optimizers.Lookahead(
            tf.keras.optimizers.Adam(learning_rate=scheduler),
            10,
            0.5,
        ),
        "dynamic",
    )
    
    model.compile(
        optimizer=optimizer,
        metrics=[TFLWLRAP(num_classes=24, name="lwlrap")],
        metric_loss_fn=NpairsLoss(temperature=0.1, name="n_pairs"),
        classification_loss_fn=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    )   

    print(' -> Training Model \n')

    callbacks = get_callbacks(pretrained_with_contrastive, fold_idx, saved_path=saved_path)
    steps_per_epoch = int((len(fold_train_dict)) / balanced_train_data_loader.batch_size)
    
    model.fit(
        balanced_train_data_loader,
        steps_per_epoch=steps_per_epoch,
        epochs=60,
        validation_data=valid_data_loader,
        callbacks=callbacks,
        verbose=2
    )

In [10]:
DEBUG = False
pretrained_folder = "../logs/2021-01-28/1/"   # simple
pretrained_folder = "../logs/2021-01-30/42/"  # mixstyle + iunterpolate + cbam
pretrained_folder = "../logs/2021-01-31/8/"  # mixstyle + iunterpolate + cbam without relu
log_folder = "../logs/"

In [11]:
backbones = [
    "efficientnetb0",
    "efficientnetb1",
    "efficientnetb2",
    "efficientnetb3",
    "efficientnetb4",
    "resnet50",
]

In [12]:
if not DEBUG:
    log_folder = prepare_log_folder(LOG_PATH)
    print(f'Logging results to {log_folder}')
    create_logger(directory=log_folder, name="logs.txt")
    
for fold_idx in range(5):
    tf.keras.backend.clear_session()
    print(f"\n-------------   Fold {fold_idx + 1} / {5}  -------------\n")
    
    main(
        fold_idx, 
        log_folder, 
        pretrained_folder + f"pretrained_best_fold{fold_idx}.h5", 
        pretrained_with_contrastive=False
    )
    
    if DEBUG:
        break

Logging results to ../logs/2021-01-31/14/

-------------   Fold 1 / 5  -------------

 -> Preparing Data 

 -> Preparing Model 

 -> Loading weights from ../logs/2021-01-31/8/pretrained_best_fold0.h5

 -> Training Model 

Epoch 1/60
Instructions for updating:
Use fn_output_signature instead
30/30 - 10s - loss: 0.0730 - lwlrap: 0.5469 - val_loss: 0.0821 - val_lwlrap: 0.7128
Epoch 2/60
30/30 - 8s - loss: 0.0667 - lwlrap: 0.7439 - val_loss: 0.0697 - val_lwlrap: 0.6979
Epoch 3/60
30/30 - 8s - loss: 0.0538 - lwlrap: 0.7060 - val_loss: 0.0716 - val_lwlrap: 0.7001
Epoch 4/60
30/30 - 8s - loss: 0.0436 - lwlrap: 0.7766 - val_loss: 0.0715 - val_lwlrap: 0.7104
Epoch 5/60
30/30 - 8s - loss: 0.0334 - lwlrap: 0.8058 - val_loss: 0.0527 - val_lwlrap: 0.7283
Epoch 6/60
30/30 - 8s - loss: 0.0274 - lwlrap: 0.8369 - val_loss: 0.0524 - val_lwlrap: 0.7538
Epoch 7/60
30/30 - 8s - loss: 0.0229 - lwlrap: 0.8557 - val_loss: 0.0587 - val_lwlrap: 0.7711
Epoch 8/60
30/30 - 8s - loss: 0.0198 - lwlrap: 0.8534 - val_

Epoch 26/60
30/30 - 8s - loss: 0.0060 - lwlrap: 0.9927 - val_loss: 0.0359 - val_lwlrap: 0.8668
Epoch 27/60
30/30 - 8s - loss: 0.0057 - lwlrap: 0.9946 - val_loss: 0.0345 - val_lwlrap: 0.8792
Epoch 28/60
30/30 - 8s - loss: 0.0073 - lwlrap: 0.9924 - val_loss: 0.0389 - val_lwlrap: 0.8506
Epoch 29/60
30/30 - 8s - loss: 0.0067 - lwlrap: 0.9967 - val_loss: 0.0407 - val_lwlrap: 0.8339
Epoch 30/60
30/30 - 8s - loss: 0.0069 - lwlrap: 0.9944 - val_loss: 0.0418 - val_lwlrap: 0.8241
Epoch 31/60
30/30 - 8s - loss: 0.0035 - lwlrap: 0.9977 - val_loss: 0.0440 - val_lwlrap: 0.8553
Epoch 32/60
30/30 - 8s - loss: 0.0076 - lwlrap: 0.9949 - val_loss: 0.0486 - val_lwlrap: 0.8265
Epoch 33/60


KeyboardInterrupt: 

[autoreload of backbones failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/usr/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/usr/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/tf/notebooks/src/backbones/__init__.py", line 12, in <module>
    from backbones.resnext import ResNeXt50
  File "/tf/notebooks/src/backbones/resnext.py", line 2, in <module>
    from tf2_resnets.resnet import ResNet
  File "/usr/lo